In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = ""
path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/'
os.chdir(path)
import time
FOLDERNAME = "LAW-OpenNMT-bert-0.85-" + str(time.strftime("%Y%m%d-%H%M"))
!mkdir $FOLDERNAME

path = path + FOLDERNAME
os.chdir(path)
!pwd

# import os
# path = '/content/drive/Shared drives/chinh-share/nmt-v3/OpenNMT-TED-EM-bert-ratio-8-2-2-20210128-0637'
# os.chdir(path)
# !pwd

/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/LAW-OpenNMT-bert-0.85-20210411-0146


In [3]:
!nvidia-smi

Sun Apr 11 01:46:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Install libraries**

In [4]:
!pip install OpenNMT-py==1.2.0
!pip install -U scikit-learn

     |████████████████████████████████| 204kB 8.0MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 2.6MB 9.8MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=cfbfe7a471e3c597e919893dad37dd7882f9db0747ea82acb3a7bfe08f5708d5
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2f70015f92025e0af663c595146083f3d9aa05
Successfully built configargparse
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 22.3MB 1.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [5]:
!wget https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/law_bert.tar.gz
!mkdir data_bin
!tar -xvf 'law_bert.tar.gz'

--2021-04-11 01:46:23--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/law_bert.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4494433 (4.3M) [application/octet-stream]
Saving to: ‘law_bert.tar.gz’

law_bert.tar.gz     100%[===================>]   4.29M  --.-KB/s    in 0.09s   

2021-04-11 01:46:23 (46.8 MB/s) - ‘law_bert.tar.gz’ saved [4494433/4494433]

en_train_EM_score_0.95
vi_valid
en_train_EM_0.95
en_train_EM_factor_0.85
en_train_EM_score_0.8
vi_train
en_train_EM_factor_0.8
en_train_EM_0.8
en_valid
en_train_EM_factor_0.95
en_train
en_train_EM_score_0.85
vi_test
en_train_EM_0.85
en_train_EM_score_0.9
en_test
en_train_EM_factor_0.9
en_train_EM_0.9


In [6]:
!mkdir -p output
!onmt_preprocess -train_src 'en_train_EM_0.85' \\
-train_tgt 'vi_train' \\
-valid_src 'en_valid' \\
-valid_tgt 'vi_valid' \\
-save_data 'output/en-vi' 

[2021-04-11 01:46:27,658 INFO] Extracting features...
[2021-04-11 01:46:27,660 INFO]  * number of source features: 0.
[2021-04-11 01:46:27,660 INFO]  * number of target features: 0.
[2021-04-11 01:46:27,660 INFO] Building `Fields` object...
[2021-04-11 01:46:27,660 INFO] Building & saving training data...
[2021-04-11 01:46:27,703 INFO] Building shard 0.
[2021-04-11 01:46:28,235 INFO]  * saving 0th train data shard to output/en-vi.train.0.pt.
[2021-04-11 01:46:28,683 INFO]  * tgt vocab size: 5120.
[2021-04-11 01:46:28,699 INFO]  * src vocab size: 13792.
[2021-04-11 01:46:28,781 INFO] Building & saving validation data...
[2021-04-11 01:46:28,821 INFO] Building shard 0.
[2021-04-11 01:46:28,856 INFO]  * saving 0th valid data shard to output/en-vi.valid.0.pt.


In [7]:
!mkdir -p model
!onmt_train -data 'output/en-vi' \\
-save_model 'model/en-vi' \\
-layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 \\
-encoder_type transformer -decoder_type transformer -position_encoding \\
-train_steps 30000  -max_generator_batches 2 -dropout 0.1 -batch_size 4096 \\
-batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 \\
-decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 \\
-param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 \\
-report_every 1000 -world_size 1 -gpu_ranks 0

[2021-04-11 01:46:30,730 INFO]  * src vocab size = 13792
[2021-04-11 01:46:30,730 INFO]  * tgt vocab size = 5120
[2021-04-11 01:46:30,730 INFO] Building model...
[2021-04-11 01:46:37,168 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(13792, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_featu

In [8]:
!ls -al model model/

model:
total 20472390
-rw------- 1 root root 698790463 Apr 11 03:03 en-vi_step_10000.pt
-rw------- 1 root root 698790463 Apr 11 01:54 en-vi_step_1000.pt
-rw------- 1 root root 698790463 Apr 11 03:10 en-vi_step_11000.pt
-rw------- 1 root root 698790463 Apr 11 03:18 en-vi_step_12000.pt
-rw------- 1 root root 698790463 Apr 11 03:26 en-vi_step_13000.pt
-rw------- 1 root root 698790463 Apr 11 03:33 en-vi_step_14000.pt
-rw------- 1 root root 698790463 Apr 11 03:41 en-vi_step_15000.pt
-rw------- 1 root root 698790463 Apr 11 03:49 en-vi_step_16000.pt
-rw------- 1 root root 698790463 Apr 11 03:56 en-vi_step_17000.pt
-rw------- 1 root root 698790463 Apr 11 04:04 en-vi_step_18000.pt
-rw------- 1 root root 698790463 Apr 11 04:11 en-vi_step_19000.pt
-rw------- 1 root root 698790463 Apr 11 04:19 en-vi_step_20000.pt
-rw------- 1 root root 698790463 Apr 11 02:01 en-vi_step_2000.pt
-rw------- 1 root root 698790463 Apr 11 04:27 en-vi_step_21000.pt
-rw------- 1 root root 698790463 Apr 11 04:34 en-vi_step

In [9]:
!onmt_translate -model model/en-vi_step_30000.pt -src en_test -tgt vi_test -output predict.txt

[2021-04-11 05:36:10,800 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:19.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [135], which does not match the required output shape [27, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an o

In [10]:
!tail vi_test

Việc giải quyết tranh chấp lao động do cơ quan, tổ chức, cá nhân có thẩm quyền giải quyết tranh chấp lao động tiến hành sau khi một trong hai bên có đơn yêu cầu do một trong hai bên từ chối thương lượng, thương lượng nhưng không thành hoặc thương lượng thành nhưng một trong hai bên không thực hiện.
Việc giám định do cá nhân hoặc do tập thể thực hiện.Giám định bổ sung
Bầu, miễn nhiệm, bãi nhiệm Chủ tịch Hội đồng thành viên; quyết định bổ nhiệm, miễn nhiệm, bãi nhiệm, ký và chấm dứt hợp đồng đối với Giám đốc hoặc Tổng giám đốc, Kế toán trưởng và người quản lý khác quy định tại Điều lệ công ty;
Người yêu cầu Tòa án Việt Nam công nhận và cho thi hành hoặc không công nhận tại Việt Nam bản án, quyết định dân sự của Tòa án nước ngoài; công nhận và cho thi hành phán quyết của Trọng tài nước ngoài phải nộp lệ phí theo quy định của pháp luật Việt Nam.
Họ, tên Thẩm phán, Thư ký phiên tòa, Kiểm sát viên; họ, tên Thẩm phán, Kiểm sát viên dự khuyết (nếu có);
Giới hạn trách nhiệm đối với người cứu hộ

In [11]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 17114 (delta 7), reused 8 (delta 4), pack-reused 17085
Receiving objects: 100% (17114/17114), 273.05 MiB | 23.57 MiB/s, done.
Resolving deltas: 100% (12323/12323), done.
Checking out files: 100% (228/228), done.


In [12]:
!ls -al

total 22475
drwx------  2 root root    4096 Apr 11 01:46 data_bin
-rw-------  1 root root  257314 Apr 11 01:32 en_test
-rw-------  1 root root 2096540 Apr 11 01:32 en_train
-rw-------  1 root root 2564912 Apr 11 01:35 en_train_EM_0.8
-rw-------  1 root root 2224075 Apr 11 01:35 en_train_EM_0.85
-rw-------  1 root root 2137239 Apr 11 01:35 en_train_EM_0.9
-rw-------  1 root root 2108457 Apr 11 01:35 en_train_EM_0.95
-rw-------  1 root root  801412 Apr 11 01:35 en_train_EM_factor_0.8
-rw-------  1 root root  688124 Apr 11 01:35 en_train_EM_factor_0.85
-rw-------  1 root root  659440 Apr 11 01:35 en_train_EM_factor_0.9
-rw-------  1 root root  650068 Apr 11 01:35 en_train_EM_factor_0.95
-rw-------  1 root root  148555 Apr 11 01:35 en_train_EM_score_0.8
-rw-------  1 root root  148555 Apr 11 01:35 en_train_EM_score_0.85
-rw-------  1 root root  148555 Apr 11 01:35 en_train_EM_score_0.9
-rw-------  1 root root  148555 Apr 11 01:35 en_train_EM_score_0.95
-rw-------  1 root root  265528 Apr 1

In [13]:
!perl OpenNMT-py/tools/multi-bleu.perl vi_test < predict.txt

BLEU = 39.57, 69.2/55.7/45.9/39.0 (BP=0.772, ratio=0.795, hyp_len=40753, ref_len=51293)
